<a href="https://colab.research.google.com/github/negodinaarina/data_visualisation/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 файлам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** c - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


# Задание 1 (3 балла)

1. Возьмите 10 000 случайных строк из таблицы sales_train и найдите в них долю товаров из категории 37. 
2. Найдите самый удачный по общему количеству продаж день для магазина St. Petersburg Nevsky Center shopping center за 2014 год.
3. Есть ли товар, который продается успешнее всех остальных (по ежемесячному количеству) во всех магазинах?

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
merged = pd.merge(sales, items).sample(10000)
print(f"Доля товаров с категорией 37 в 10 000 случайных строк - {len(merged[merged['category_id']==37])/10000}")


Доля товаров с категорией 37 в 10 000 случайных строк - 0.0652


In [ ]:
shops = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/shops.csv'))
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))
id = shops[shops.shop_name == ' St. Petersburg Nevsky Center shopping center']['shop_id']
sales['date'] = pd.to_datetime(sales['date'], dayfirst=True)
sales14 = (sales["date"] > '01-01-2014') & (sales["date"] <= '31-12-2014') & (sales["shop_id"] == int(id))
sales14 = sales.loc[sales14]
sales14 = sales14[['date', 'item_cnt_day']].groupby(['date']).sum().reset_index()
print(f"Самый удачный по общему количеству продаж день для магазина St. Petersburg Nevsky Center -{sales14.nlargest(1, 'item_cnt_day')['date']}")

Самый удачный по общему количеству продаж день для магазина St. Petersburg Nevsky Center -359   2014-12-27
Name: date, dtype: datetime64[ns]


/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:52: UserWarning: Parsing '31-12-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.le)


In [ ]:
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
sales_g = sales[['date_block_num', 'item_id', 'item_cnt_day']].groupby(["date_block_num", "item_id"]).sum().reset_index()
sales_g = sales_g.sort_values(by='item_cnt_day', ascending=False)
sales_g.drop_duplicates(subset=['date_block_num'])
l = list(sales_g['item_id'])
f = True
for i in range(1, len(l)):
  if l[i-1] != l[i]:
    f = False
    break
if f:
  print(f'Есть товар, который продавался успешнее всех остальных каждый месяц - {l[0]}')
else:
  print('Такого товара нет')
# print(sales_g.drop_duplicates(subset=['date_block_num']))


Такого товара нет


# Задание 2 (2 балла)

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))[['item_id', 'item_cnt_day']].groupby(['item_id']).sum().reset_index().sort_values(by='item_cnt_day', ascending=False).head(10)
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
print('Самые продающиеся товары:')
for item in sales.item_id:
  print(items.loc[items['item_id'] == item]['item_name'].values[0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Самые продающиеся товары:
Corporate package T-shirt 1C Interest white (34 * 42) 45 microns
Diablo III [PC, Jewel, Russian version]
Grand Theft Auto V [PS3, Russian subtitles]
Acceptance of funds for 1C-Online
Playstation Store replenishment of wallet: Payment card 1000 rubles.
Grand Theft Auto V [Xbox 360, Russian subtitles]
Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1108A/B01)
Grand Theft Auto V [PC, Russian subtitles]
Battlefield 4 [PC, Russian version]
Some of us [PS3, Russian version]


In [ ]:
ids = sales['item_id']
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))[['date', 'item_id']]
sales['date'] = pd.to_datetime(sales['date'], dayfirst=True)
merged = pd.merge(ids, sales)
merged = merged.groupby(merged.date.dt.year)['item_id'].apply(set).reset_index(name="item_id")


In [ ]:
set2013 = merged.at[0, 'item_id']
set2014 = merged.at[1, 'item_id']
set2015 = merged.at[2, 'item_id']
ids = set2013.intersection(set2014, set2015)
for id in ids:
  print(items.loc[items['item_id'] == id]['item_name'].values[0])
# print(ids)

Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1108A/B01)
Grand Theft Auto V [PS3, Russian subtitles]
Acceptance of funds for 1C-Online
Corporate package T-shirt 1C Interest white (34 * 42) 45 microns
Grand Theft Auto V [Xbox 360, Russian subtitles]
Diablo III [PC, Jewel, Russian version]
Some of us [PS3, Russian version]
Playstation Store replenishment of wallet: Payment card 1000 rubles.
Battlefield 4 [PC, Russian version]


# Задание 3 (3 балла)

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

*Если таких категорий несколько, выведите все.*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
cats = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/item_categories.csv'))
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))[['item_id', 'item_cnt_day']].groupby(["item_id"]).sum().reset_index()
sales['category'] = 0
for item in sales.item_id:
  cat_id = items[items.item_id == item]['category_id'].values[0]
  sales.loc[sales['item_id'] == item, ['category']] = cat_id
sales = sales[['category', 'item_cnt_day']].groupby(['category']).sum().reset_index()
sales = sales.sort_values(by='item_cnt_day', ascending=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sales = sales.reset_index(drop=True)
minc = sales.loc[len(sales) - 1, 'item_cnt_day']
maxc = sales.loc[0, 'item_cnt_day']
max_c = []
min_c = []
max_c.append(sales.at[0, 'category'])
min_c.append(sales.at[len(sales) - 1, 'category'])
for i in range(1, len(sales) - 1):
  if sales.loc[i, 'item_cnt_day'] == maxc:
    max_c.append(sales.loc[i, 'category'])

for i in range(len(sales) - 2, 0, -1):
  if sales.loc[i, 'item_cnt_day'] == minc:
    min_c.append(sales.loc[i, 'category'])

print('Самые продаваемые категории:')
for id in max_c:
  name = cats.loc[cats['item_category_id'] == id]['item_category_name'].values[0]
  print(f'Категория товара - {name}, id - {id}')

print('Самые непродаваемые категории:')
for id in min_c:
  name = cats.loc[cats['item_category_id'] == id]['item_category_name'].values[0]
  print(f'Категория товара - {name}, id - {id}')


Самые продаваемые категории:
Категория товара -  Cinema - DVD, id - 40
Самые непродаваемые категории:
Категория товара -  Books - Cognitive literature, id - 51
Категория товара -  Game consoles - PS2, id - 10


# Задание 4 (3 балла)

1. Для категорий из задания 3 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами. Оставьте в таблице только те товары, среднее количество продаж которых строго больше 100.
2. Найдите месяц с максимальными продажами для каждого товара из пункта 1.


In [ ]:
cats = max_c + min_c
df = pd.DataFrame(cats, columns=['category_id'])
sales = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/sales_train.csv'))
items = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/data MEVD/items.csv'))
df = pd.merge(sales, items)
iwannasurvive = df.loc[df['category_id']. isin(cats)]
top_shops = iwannasurvive.groupby(['shop_id'])['item_cnt_day'].sum().reset_index().sort_values(by='item_cnt_day', ascending=False).head(5)
top5 = df.loc[df['shop_id']. isin(top_shops['shop_id']) & df['category_id']. isin(cats)].groupby(['item_id', 'shop_id'])['item_cnt_day'].sum().reset_index()
res = top5.groupby('item_id')['item_cnt_day'].nlargest(5).groupby(level=0).mean().reset_index()
res = res[res.item_cnt_day > 100]
res



,item_id,item_cnt_day
23,30,161.6
24,32,167.4
316,8415,107.8
801,9781,187.2
844,9862,117.6
910,9983,162.4
1000,10292,156.8
1002,10298,272.6
1044,10376,140.0
1104,10487,138.8


In [ ]:
merged = sales[sales.item_id.isin(res.item_id)].groupby(['item_id', 'date_block_num'])['item_cnt_day'].sum().reset_index()
rows_ids = merged.groupby(['item_id'])['item_cnt_day'].idxmax().reset_index()
res = merged.loc[rows_ids.item_cnt_day, ['item_id', 'date_block_num']]
res

,item_id,date_block_num
0,30,1
33,32,0
67,8415,14
78,9781,8
96,9862,14
117,9983,9
134,10292,8
161,10298,10
186,10376,12
196,10487,11


# Задание 5 (3 балла)

1. Для категорий из задания 3 найдите среднюю и медианную цену по категории в пределах каждого месяца. Выведите топ-10 товаров по медиане из каждой категории.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
# average price per month
lordplssaveme = pd.merge(sales, items).loc[df['category_id']. isin(cats)]
lordplssaveme.groupby(['item_id', 'date_block_num'])['item_price'].mean()

item_id  date_block_num
0        20                 58.000000
2        19                 58.000000
         22                 58.000000
3        18                100.000000
         19                 58.000000
                              ...    
22162    32                349.000000
         33                349.000000
22163    31                163.270385
         32                169.000000
         33                162.698400
Name: item_price, Length: 53616, dtype: float64

In [ ]:
# median price per month - top 10
lordplssaveme.groupby(['item_id', 'date_block_num', 'category_id'])['item_price'].median().reset_index().sort_values('item_price', ascending=False).reset_index().groupby(['date_block_num', 'category_id']).head(10).sort_values(['date_block_num', 'item_price'], ascending=[True, False])

,index,item_id,date_block_num,category_id,item_price
6,24632,13993,0,40,999.0
13,21504,12686,0,40,999.0
29,8534,9746,0,40,849.0
44,7672,9601,0,40,849.0
61,7934,9640,0,40,849.0
...,...,...,...,...,...
221,28537,14711,33,40,699.0
328,48882,20214,33,40,649.0
428,21535,12686,33,40,574.0
453,18622,11922,33,40,549.0


In [ ]:
# В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.
lordplssaveme = pd.merge(sales, items)
lordplssaveme['date'] = pd.to_datetime(lordplssaveme['date'], dayfirst=True)
lordplssaveme['year'] = lordplssaveme['date'].dt.year
lordplssaveme['month'] = lordplssaveme['date'].dt.month
lordplssaveme = lordplssaveme.groupby(['year', 'month'])['item_price'].mean().reset_index()
sales2013 = lordplssaveme.loc[(lordplssaveme['year'] == 2013) | (lordplssaveme['year'] == 2014) & (lordplssaveme['month'] == 1)]
sales2014 = lordplssaveme.loc[(lordplssaveme['year'] == 2014) | (lordplssaveme['year'] == 2013) & (lordplssaveme['month'] == 12) | (lordplssaveme['year'] == 2015) & (lordplssaveme['month'] == 1)]
sales2015 = lordplssaveme.loc[(lordplssaveme['year'] == 2015) | (lordplssaveme['year'] == 2014) & (lordplssaveme['month'] == 12)]
def find_months(df):
  return df.loc[((df['item_price'] < df.shift(1)['item_price']) & (df['item_price'] < df.shift(-1)['item_price']))]

,year,month,item_price
25,2015,2,1010.465368
31,2015,8,951.374139


In [ ]:
find_months(sales2013)


,year,month,item_price
4,2013,5,624.179993
6,2013,7,618.578247


In [ ]:
find_months(sales2014)

,year,month,item_price
12,2014,1,862.869128
15,2014,4,852.160167
18,2014,7,847.046420


In [ ]:
find_months(sales2015)

,year,month,item_price
25,2015,2,1010.465368
31,2015,8,951.374139
